# Analysing rental store data

## Storing data into dictionary of dfs

In [ ]:
import duckdb

dfs= {}

with duckdb.connect("data/sakila.duckdb") as conn:
    description = conn.sql("DESC;").df()
    for name in description["name"]:
        if name[0] != '_':
            dfs[name] = conn.sql(f"FROM staging.{name};").df()

            duckdb.register(name, dfs[name])

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,_dlt_load_id,_dlt_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,<NA>,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2021-03-06 15:52:00,1765184285.64651,Mk/ecnZ0CZTa3Q
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,<NA>,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2021-03-06 15:52:00,1765184285.64651,BzY69k73XftSzw
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,<NA>,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2021-03-06 15:52:00,1765184285.64651,ONCELr5sCEcqIw
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,<NA>,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2021-03-06 15:52:00,1765184285.64651,nADNJSI9fUzj/g
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,<NA>,6,2.99,130,22.99,G,Deleted Scenes,2021-03-06 15:52:00,1765184285.64651,jLxXiNV6/i4QCA


## Task 1 - EDA in python

a) Which movies are longer then 3 hours (180 minutes), title and length

In [7]:
duckdb.sql("""--sql
           SELECT title, length
           FROM film
           WHERE length >= 180
           ORDER BY length
""").df().head()

,title,length
0,ALLEY EVOLUTION,180
1,CONFIDENTIAL INTERVIEW,180
2,SOMETHING DUCK,180
3,NASH CHOCOLAT,180
4,IMPACT ALADDIN,180


b) Which movies have the workd "love" in its title? showing following columns
- title
- rating
- length 
- description

In [47]:
## I used chatgpt after I wrote one \ first and didn't realize i need two in order for it to escape correctly
duckdb.sql("""--sql
        SELECT
           title,
           rating,
           length,
           description
        FROM film
        WHERE
           regexp_matches(title, '\\bLOVE\\b')
""").df()

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
3,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
4,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...


d) The rental rate is the cost to rent a movie and the rental duration is the number of days a customer can keep the movie. The Manager wants to know the 10 most expensive movies to rent per day.


In [46]:
duckdb.sql("""--sql
        SELECT
           title,
           rental_rate,
           rental_duration,
           rental_rate / rental_duration as cost_per_day
        FROM film
        ORDER BY cost_per_day DESC;
""").df().head(62)

,title,rental_rate,rental_duration,cost_per_day
0,PATHS CONTROL,4.99,3,1.663333
1,BACKLASH UNDEFEATED,4.99,3,1.663333
2,BEHAVIOR RUNAWAY,4.99,3,1.663333
3,VIRTUAL SPOILERS,4.99,3,1.663333
4,TEEN APOLLO,4.99,3,1.663333
...,...,...,...,...
57,PARK CITIZEN,4.99,3,1.663333
58,SUIT WALLS,4.99,3,1.663333
59,BEAST HUNCHBACK,4.99,3,1.663333
60,SUNRISE LEAGUE,4.99,3,1.663333


NOTE: There are 62 movies that all cost the same to rent per day

e) Which actors have played in most movies? Show the top 10 actors with the number of movies they have
played in.


In [48]:
films_joined = duckdb.sql("""--sql
    SELECT 
        a.first_name || ' ' || a.last_name AS actor,
        a.actor_id::INT as actor_id,
        f.title,
        f.description,
        f.rating,
        f.release_year,
        f.rental_duration,
        f.length,
        c.name AS category
    FROM film f
        LEFT JOIN film_actor fa ON f.film_id = fa.film_id
        LEFT JOIN actor a ON a.actor_id = fa.actor_id
        LEFT JOIN film_category fc ON fc.film_id = f.film_id
        LEFT JOIN category c ON fc.category_id = c.category_id
""")

duckdb.sql("""--sql
    SELECT 
        actor,
        COUNT(*) AS number_films
    FROM films_joined
    GROUP BY actor
    ORDER BY number_films DESC
""").df().head(10)



,actor,number_films
0,SUSAN DAVIS,54
1,GINA DEGENERES,42
2,WALTER TORN,41
3,MARY KEITEL,40
4,MATTHEW CARREY,39
5,SANDRA KILMER,37
6,SCARLETT DAMON,36
7,GROUCHO DUNST,35
8,ANGELA WITHERSPOON,35
9,VAL BOLGER,35


f) Now it's time for you to choose your own question to explore the sakila database! Write down 3-5
questions you want to answer and then answer them using pandas and duckdb.


Top and bottom 10 movies replacement cost.

## Task 2 - graphs

a) Who are our top 5 customers by total spend? The Manager wants to know so that they can reward them
with special offers. Create a bar chart showing the top 5 customers by total spend.


b) How much money does each film category bring in? Make a bar chart showing total revenue per film
category.
